In [2]:





import re
from requests import get
import requests
import os
import dotenv
dotenv.load_dotenv()

ACCESS_TOKEN = os.getenv("OPEN_CITATION_API_KEY")


API_CALL = "https://opencitations.net/api/v1/metadata/10.1108/jd-12-2013-0166"
HTTP_HEADERS = {"authorization": "94762995-8420-4272-a366-8b7aad4c40f8"}

get(API_CALL, headers=HTTP_HEADERS)
print(get(API_CALL, headers=HTTP_HEADERS).json())

base_url = "https://opencitations.net/index/api/v1"


[{'citation': '10.1057/s41275-017-0070-x; 10.1016/j.techsoc.2018.03.005; 10.1007/978-3-319-73165-0_24; 10.1007/978-3-030-00668-6_21; 10.1007/978-3-319-17966-7_10; 10.1007/978-3-319-90548-8_7; 10.1007/978-3-030-01379-0_9; 10.59350/zcrhs-r7h02; 10.1101/108480; 10.7554/elife.32822; 10.1145/3197026.3197050; 10.3233/sw-160224; 10.3233/sw-180307; 10.3233/ds-190016; 10.3233/ds-190019; 10.7717/peerj-cs.421; 10.3346/jkms.2015.30.11.1545; 10.1007/978-3-030-30796-7_8; 10.1007/978-3-030-30796-7_31; 10.1093/bib/bbx057; 10.1371/journal.pone.0238801; 10.1177/0961000615616450; 10.1142/s021964921850034x; 10.3390/app12063203; 10.1007/978-3-319-58694-6_23; 10.1162/qss_a_00183; 10.1186/s12859-019-2607-x; 10.2139/ssrn.3081354; 10.35668/2520-6524-2019-4-10; 10.59350/kgz48-bt453; 10.59350/8pkca-5dx66; 10.1007/978-3-319-68204-4_19; 10.1007/978-3-030-61244-3_16; 10.1109/mcse.2019.2952840', 'reference': '10.1136/bmj.b2680; 10.1145/1217821.1226695; 10.1145/2362499.2362502; 10.1136/bmj.a568; 10.1145/2494266.24942

In [44]:

def get_referances(doi):
    # Endpoint for retrieving metadata
    endpoint = f"{base_url}/metadata/{doi}"

    # Make a GET request to the endpoint
    response = requests.get(endpoint)
    
    if response.status_code == 200:
        
        data = response.json()
        
        references = data[0].get("reference", "")
        
        return references
    else:
        # If the request fails, print the error message
        print(f"Failed to retrieve metadata for DOI {doi}. Error: {response.text}")
        return None, None, None



In [45]:
print(get_referances("10.1108/jd-12-2013-0166"))


10.1136/bmj.b2680; 10.1145/1217821.1226695; 10.1145/2362499.2362502; 10.1136/bmj.a568; 10.1145/2494266.2494271; 10.1523/jneurosci.0003-08.2008; 10.1007/978-3-319-03524-6_29; 10.1073/pnas.0407743101; 10.1007/bf02457980; 10.1016/j.websem.2012.08.001; 10.1016/j.websem.2013.05.001; 10.1002/(sici)1097-4571(198909)40:5<342::aid-asi7>3.0.co;2-u; 10.1007/s11192-009-0021-2; 10.1002/asi.4630240406; 10.1145/1816123.1816198; 10.1108/jd-07-2012-0082; 10.1038/493159a; 10.1126/science.149.3683.510; 10.1038/495437a; 10.1042/bj20091474; 10.1371/journal.pcbi.1000361; 10.1101/sqb.1972.036.01.015; 10.1038/502295a; 10.1038/502298a; 10.1001/jama.295.1.90; 10.5210/fm.v2i4.522; 10.1525/bio.2010.60.5.2; 10.1177/030631277500500106; 10.5860/crln.73.10.8846; 10.1007/s10579-012-9211-2; 10.1087/2009202; 10.1038/nature28042; 10.1177/030631277400400102; 10.7717/peerj.175; 10.1371/journal.pone.0000308; 10.5539/ass.v9n5p18; 10.1371/journal.pcbi.0010034; 10.1371/journal.pntd.0000228; 10.1007/978-3-540-89704-0_17; 10.311

In [46]:

def get_author(doi):
    # Endpoint for retrieving metadata
    endpoint = f"{base_url}/metadata/{doi}"

    # Make a GET request to the endpoint
    response = requests.get(endpoint)
    
    if response.status_code == 200:
        
        data = response.json()
        
        author = data[0].get("author", "")
        
        # Initialize lists to store names and IDs
        names = []
        ids = []

        # Split author data by semicolon to get individual authors
        authors = author.split(";")

        # Iterate over each author
        for author in authors:
            # Split the author into name and ID
            parts = author.strip().split(",")
            if len(parts) >= 2:
                name = parts[0].strip() + " " + parts[1].strip()  # Concatenate first name and last name
                names.append(name.strip())

                if len(parts) == 3:
                    author_id = parts[2].strip()
                    ids.append(author_id)
                else:
                    ids.append(None)  # If ID doesn't exist, append None
            else:
                # If no ID is provided, append None and handle names
                names.append(parts[0].strip())
                ids.append(None)
        return names, ids
    else:
        # If the request fails, print the error message
        print(f"Failed to retrieve metadata for DOI {doi}. Error: {response.text}")
        return None, None, None




In [48]:
print(get_author("10.1108/jd-12-2013-0166"))


(['Peroni Silvio', 'Dutton Alexander', 'Gray Tanya', 'Shotton David'], ['0000-0003-0530-4305', '0000-0003-1448-3114', None, None])


In [49]:
def create_record(doi):
    DOI = doi
    authors, author_ids = get_author(DOI)
    references = get_referances(DOI)
    record =  {
        "DOI": DOI,
        "authors": authors,
        "author_ids": author_ids,
        "references": (references)
    }
    return record




In [38]:
test_record = create_record("10.1108/jd-12-2013-0166")


In [50]:

print(test_record['DOI'])
print(test_record['authors'])
print(test_record['author_ids'])
print(test_record['references'])




10.1108/jd-12-2013-0166
['Peroni Silvio', 'Dutton Alexander', 'Gray Tanya', 'Shotton David']
['0000-0003-0530-4305', '0000-0003-1448-3114', None, None]
10.1136/bmj.b2680; 10.1145/1217821.1226695; 10.1145/2362499.2362502; 10.1136/bmj.a568; 10.1145/2494266.2494271; 10.1523/jneurosci.0003-08.2008; 10.1007/978-3-319-03524-6_29; 10.1073/pnas.0407743101; 10.1007/bf02457980; 10.1016/j.websem.2012.08.001; 10.1016/j.websem.2013.05.001; 10.1002/(sici)1097-4571(198909)40:5<342::aid-asi7>3.0.co;2-u; 10.1007/s11192-009-0021-2; 10.1002/asi.4630240406; 10.1145/1816123.1816198; 10.1108/jd-07-2012-0082; 10.1038/493159a; 10.1126/science.149.3683.510; 10.1038/495437a; 10.1042/bj20091474; 10.1371/journal.pcbi.1000361; 10.1101/sqb.1972.036.01.015; 10.1038/502295a; 10.1038/502298a; 10.1001/jama.295.1.90; 10.5210/fm.v2i4.522; 10.1525/bio.2010.60.5.2; 10.1177/030631277500500106; 10.5860/crln.73.10.8846; 10.1007/s10579-012-9211-2; 10.1087/2009202; 10.1038/nature28042; 10.1177/030631277400400102; 10.7717/peerj.

In [54]:
import csv
def write_record_to_csv(record, filename):
    DOI = record['DOI']
    authors = record['authors']
    author_ids = record['author_ids']
    references = record['references']

    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the heade
        # Write each record
        writer.writerow([DOI, authors, author_ids, references])
    

write_record_to_csv(test_record, "test.csv")

In [62]:
import csv

def print_header():
    # Open the file in append mode and keep it open with the 'with' statement
    with open("test.csv", mode='w', newline='') as file:
        # Create a CSV writer object
        writer = csv.writer(file)


        # Write the row
        writer.writerow(['DOI', 'Author', 'Author ID', 'Reference'])


In [64]:
def run(doi_list):
    for doi in doi_list:
        record = create_record(doi)
        write_record_to_csv(record, "test.csv")
        


In [63]:
print_header()